In [1]:
import build_users_data as build
import projects 
import facilities
import users_and_facilities
import users_and_projects

import pandas as pd

In [8]:
# reporting_periods = ['2013-2014', '2015-2017', '2018-2020', '2021']
# mrr_data_years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
reporting_periods = ['2018-2020']
mrr_data_years = ['2018', '2019', '2020']

compliance_report_path = '../data/compliance-reports/'
mrr_data_path = '../data/mrr-data/'
issuance_table_path = '../data/'

In [3]:
mrr_file_year = {
    '2013': '2019',
    '2014': '2019',
    '2015': '2019',
    '2016': '2020',
    '2017': '2020',
    '2018': '2020',
    '2019': '2020',
    '2020': '2021',
    '2021': '2022'
}
reporting_periods = {
    '2013': '2013-2014',
    '2014': '2013-2014',
    '2015': '2015-2017',
    '2016': '2015-2017',
    '2017': '2015-2017',
    '2018': '2018-2020',
    '2019': '2018-2020',
    '2020': '2018-2020',
    '2021': '2021'
}

In [4]:
project_df = projects.read_project_data(issuance_table_path)

In [5]:
facility_df = facilities.read_facility_data(mrr_data_path, mrr_data_years)

/srv/conda/envs/notebook/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:315: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [11]:
data_path = compliance_report_path

user_project_df = pd.DataFrame()
for reporting_period in reporting_periods:
    df = pd.read_excel(
        data_path + reporting_period + 'compliancereport.xlsx',
        sheet_name=reporting_period + ' ' + 'Offset Detail',
        skiprows=4,
    )
    df = df[~pd.isnull(df).any(axis=1)]
    df['reporting_period'] = reporting_period
    user_project_df = user_project_df.append(df)
rename_d = {
    'Entity ID': 'user_id',
    'Quantity': 'quantity',
    'ARB Project ID #': 'arb_id',
}
user_project_df.rename(
    columns=rename_d,
    inplace=True,
)
user_project_df = user_project_df[['user_id', 'arb_id', 'quantity', 'reporting_period']]
user_project_df['user_id'] = user_project_df['user_id'].str.strip()
user_project_df['arb_id'] = user_project_df['arb_id'].str.strip()

# ignoring offset vintage lets us simplify arb_id and collapse
# rows that had the same entity and project but different vintages
user_project_df['arb_id'] = user_project_df['arb_id'].str.split('-').apply(lambda x: x[0])

user_project_df = (
    user_project_df.groupby(['user_id', 'arb_id', 'reporting_period'])['quantity']
    .sum()
    .astype(int)
    .reset_index()
)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [12]:
user_project_df

,user_id,arb_id,quantity,reporting_period
0,CA1006,CAFR5305,18723,2018-2020
1,CA1016,CAOD5013,62,2018-2020
2,CA1019,CAFR5198,12240,2018-2020
3,CA1019,CAFR5208,2291,2018-2020
4,CA1019,CAFR5229,49027,2018-2020
...,...,...,...,...
990,CA3186,CAMM5560,10000,2018-2020
991,CA3186,CAOD5608,85000,2018-2020
992,CA3186,CAOD5619,60000,2018-2020
993,CA3186,CAOD6481,1379,2018-2020


In [6]:
user_project_df = users_and_projects.read_user_project_data(compliance_report_path, reporting_periods)

FileNotFoundError: [Errno 2] No such file or directory: '../data/compliance-reports/2013compliancereport.xlsx'

In [49]:
user_facility_df = users_and_facilities.read_user_facility_data(
        compliance_report_path, reporting_periods
    )


In [50]:
user_facility_df

,user_id,user_name,facility_id,reporting_period
0,CA1000,"Oildale Energy, LLC",100891,2013-2014
1,CA1001,"The Newark Group, Inc.",104135,2013-2014
1,CA1001,"The Newark Group, Inc.",104136,2013-2014
2,CA1003,Plumas-Sierra Rural Electric Cooperative,3100,2013-2014
3,CA1006,"Frito-Lay, Inc.",101235,2013-2014
...,...,...,...,...
274,CA3371,"BB ENERGY USA, LLC",104867,2021
275,CA3376,"3 PR Trading, Inc.",104903,2021
276,CA3382,Northrop Grumman Systems Corporation,104227,2021
277,CA3386,"Corteva Agriscience, LLC",101733,2021


In [56]:
user_facility_df.loc[user_facility_df["user_id"] == "CA1279"]

,user_id,user_name,facility_id,reporting_period
119,CA1279,Phillips 66 Company,100303,2013-2014
119,CA1279,Phillips 66 Company,100329,2013-2014
119,CA1279,Phillips 66 Company,100351,2013-2014
119,CA1279,Phillips 66 Company,100913,2013-2014
119,CA1279,Phillips 66 Company,101226,2013-2014
115,CA1279,Phillips 66 Company,100303,2015-2017
115,CA1279,Phillips 66 Company,100329,2015-2017
115,CA1279,Phillips 66 Company,100351,2015-2017
115,CA1279,Phillips 66 Company,100913,2015-2017
115,CA1279,Phillips 66 Company,101226,2015-2017
